In [716]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import and read charity_data.csv
import pandas as pd 
application_df = pd.read_csv('../Resources/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# 3. Preprocess the data

In [717]:
# Drop non-beneficial ID columns
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

In [718]:
# Determine the number of unique values in each column
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [719]:
# APPLICATION_TYPE value counts for binning
application_values = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_application = application_values[application_values < 1000].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, 'Other')

In [720]:
# CLASSIFICATION value counts for binning
classification_values = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_class = classification_values[classification_values < 1000].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls, 'Other')

In [721]:
# AFFILIATION value counts for binning
affiliation_values = application_df['AFFILIATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_affiliation = affiliation_values[affiliation_values < 1000].index.tolist()

# Replace in dataframe
for aff in replace_affiliation:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(aff, 'Other')

In [722]:
# USE_CASE value counts for binning
case_values = application_df['USE_CASE'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_case = case_values[case_values < 1000].index.tolist()

# Replace in dataframe
#for case in replace_case:
#    application_df.USE_CASE = application_df.USE_CASE.replace(case, 'Other')

In [723]:
# ORGANIZATION value counts for binning
organization_values = application_df['ORGANIZATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_organization = organization_values[organization_values < 1000].index.tolist()

# Replace in dataframe
for org in replace_organization:
    application_df.ORGANIZATION = application_df.ORGANIZATION.replace(org, 'Other')

In [724]:
# Generate the categorical variable list
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [725]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

In [726]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1)

In [727]:
# Split the preprocessed data into features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [728]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# 4. Design a neural network model

In [729]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_142 (Dense)            (None, 80)                2720      
_________________________________________________________________
dense_143 (Dense)            (None, 30)                2430      
_________________________________________________________________
dense_144 (Dense)            (None, 12)                372       
_________________________________________________________________
dense_145 (Dense)            (None, 1)                 13        
Total params: 5,535
Trainable params: 5,535
Non-trainable params: 0
_________________________________________________________________


In [730]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 5. Create a callback

In [731]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('optimized_checkpoints/', exist_ok=True)
checkpoint_path = 'optimized_checkpoints/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [732]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6926 - accuracy: 0.5258
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5308
Epoch 3/100
804/804 [==============================] - 1s 958us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 4/100
804/804 [==============================] - 1s 883us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 5/100
777/804 [===========================>..] - ETA: 0s - loss: 0.6912 - accuracy: 0.5323
Epoch 00005: saving model to optimized_checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 931us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 6/100
804/804 [==============================] - 1s 900us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 7/100
804/804 [==============================] - 1s 905us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 8/100
804/804 [==============================] - 1s 920us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 9/100
804/804 [=======

804/804 [==============================] - 1s 992us/step - loss: 0.6911 - accuracy: 0.53211s - los
Epoch 63/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 64/100
804/804 [==============================] - 1s 928us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 65/100
540/804 [===================>..........] - ETA: 0s - loss: 0.6911 - accuracy: 0.5320
Epoch 00065: saving model to optimized_checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 957us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 66/100
804/804 [==============================] - 1s 925us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 67/100
804/804 [==============================] - 1s 903us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 68/100
804/804 [==============================] - 1s 904us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 69/100
804/804 [==============================] - 1s 916us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 70/100
510/804 [=

In [733]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.6854 - accuracy: 0.5529
Loss: 0.6853809356689453, Accuracy: 0.5528863072395325


# 6. Export results

In [ ]:
# Export the model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

# Random Forest

In [455]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df_applications = pd.read_csv('../Resources/charity_data.csv')

In [456]:
# Preprocess the data
df_applications = df_applications.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])
# Bin APPLICATION_TYPE, CLASSIFICATION, AFFILIATION, USE_CASE, and ORGANIZATION
application_values = df_applications['APPLICATION_TYPE'].value_counts()
replace_application = application_values[application_values < 50].index.tolist()
for app in replace_application:
    df_applications.APPLICATION_TYPE = df_applications.APPLICATION_TYPE.replace(app, 'Other')
classification_values = df_applications['CLASSIFICATION'].value_counts()
replace_class = classification_values[classification_values < 500].index.tolist()
for cls in replace_class:
    df_applications.CLASSIFICATION = df_applications.CLASSIFICATION.replace(cls, 'Other')
affiliation_values = df_applications['AFFILIATION'].value_counts()
replace_affiliation = affiliation_values[affiliation_values < 1000].index.tolist()
for aff in replace_affiliation:
    df_applications.AFFILIATION = df_applications.AFFILIATION.replace(aff, 'Other')
case_values = df_applications['USE_CASE'].value_counts()
replace_case = case_values[case_values < 1000].index.tolist()
for case in replace_case:
    df_applications.USE_CASE = df_applications.USE_CASE.replace(case, 'Other')
organization_values = df_applications['ORGANIZATION'].value_counts()
replace_organization = organization_values[organization_values < 1000].index.tolist()
for org in replace_organization:
    df_applications.ORGANIZATION = df_applications.ORGANIZATION.replace(org, 'Other')
# Merge one-hot encoded features and drop the originals
application_cat = df_applications.dtypes[df_applications.dtypes == "object"].index.tolist()
enc = OneHotEncoder(sparse=False)
encode_df = pd.DataFrame(enc.fit_transform(df_applications[application_cat]))
encode_df.columns = enc.get_feature_names(application_cat)
df_applications = df_applications.merge(encode_df, left_index=True, right_index=True)
df_applications = df_applications.drop(application_cat, 1)
df_applications.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [457]:
df_applications.columns

Index(['ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T13', 'APPLICATION_TYPE_T19',
       'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5',
       'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'APPLICATION_TYPE_T9', 'AFFILIATION_CompanySponsored',
       'AFFILIATION_Independent', 'AFFILIATION_Other', 'CLASSIFICATION_C1000',
       'CLASSIFICATION_C1200', 'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100',
       'CLASSIFICATION_C3000', 'CLASSIFICATION_C7000', 'CLASSIFICATION_Other',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Other', 'ORGANIZATION_Trust',
       'INCOME_AMT_0', 'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M'],
      dtype='object')

In [458]:
# Define the features and target sets
X = df_applications.copy()
X = X.drop('IS_SUCCESSFUL', axis=1)
y = df_applications['IS_SUCCESSFUL'].ravel()

In [459]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [460]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [461]:
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.7095043731778425


In [462]:
importances = rf_model.feature_importances_
sorted(zip(importances, X.columns), reverse=True)

[(0.4025830068786512, 'ASK_AMT'),
 (0.13282381850313302, 'AFFILIATION_CompanySponsored'),
 (0.12168446961145543, 'AFFILIATION_Independent'),
 (0.026999508289862566, 'APPLICATION_TYPE_T5'),
 (0.026624938841938044, 'ORGANIZATION_Association'),
 (0.024990002007273394, 'APPLICATION_TYPE_T10'),
 (0.020375325892822677, 'ORGANIZATION_Trust'),
 (0.01979997225751412, 'CLASSIFICATION_C2100'),
 (0.017988334219177258, 'CLASSIFICATION_C2000'),
 (0.017660280224002397, 'APPLICATION_TYPE_T19'),
 (0.01660273895265352, 'APPLICATION_TYPE_T4'),
 (0.016514316218633127, 'APPLICATION_TYPE_T3'),
 (0.013806555216777245, 'CLASSIFICATION_C1000'),
 (0.013275641225230703, 'CLASSIFICATION_C7000'),
 (0.01235793644910077, 'USE_CASE_ProductDev'),
 (0.01151696809346162, 'APPLICATION_TYPE_T6'),
 (0.010274276597811554, 'USE_CASE_Preservation'),
 (0.009476992326039848, 'CLASSIFICATION_C1200'),
 (0.009329282810696069, 'INCOME_AMT_0'),
 (0.009168459027014589, 'CLASSIFICATION_Other'),
 (0.00774055618626549, 'CLASSIFICATION_C

# Drop lower ranked features

In [49]:
# Drop lower ranked features
X = df_applications[[
    'ASK_AMT', 'AFFILIATION_CompanySponsored', 'AFFILIATION_Independent', 'APPLICATION_TYPE_T5',
    'ORGANIZATION_Association', 'APPLICATION_TYPE_T10', 'ORGANIZATION_Trust', 'CLASSIFICATION_Other',
    'CLASSIFICATION_C2100', 'CLASSIFICATION_C2000', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T4',
    'APPLICATION_TYPE_T3', 'CLASSIFICATION_C1000', 'USE_CASE_ProductDev', 'APPLICATION_TYPE_T6',
    'USE_CASE_Preservation', 'CLASSIFICATION_C1200']]
y = df_applications['IS_SUCCESSFUL'].ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
# Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# Fit the model
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
# Make predictions
predictions = rf_model.predict(X_test_scaled)
# Evaluate the model
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.707871720116618
